In [188]:
# import Pkg
# Pkg.add("DataFrames")
# Pkg.add("CSV")
# Pkg.add("IntervalSets")

In [189]:
using DataFrames, CSV
using IntervalArithmetic
using LaTeXStrings
include("./src/interval-weight-estimation.jl")
include("./src/display/display-crisp-matrix.jl")
include("./src/display/display-interval-vector.jl")
include("./src/display/display-vector.jl")
include("./src/crisp-pcm.jl")
include("./src/importance-estimation.jl")
include("./src/solve-interval-ahp.jl")

solveIntervalAHP (generic function with 1 method)

In [190]:
# 真の区間重要度を読み込み
df = CSV.File("./src/data/N5_Given_interval_weight.csv", header=false) |> DataFrame
T = Vector{Interval{Float64}}() # Float64のところにTを使うとおかしくなるのはなぜか？
for i in 1:5
    push!(T, interval(df[1, 2*i-1], df[1, 2*i]))
end

T # 真の区間重要度
# display(L"W = %$(intervalVectorLaTeXString(W))")

5-element Vector{Interval{Float64}}:
 [0.269999, 0.330001]
 [0.209999, 0.290001]
 [0.149999, 0.25]
 [0.08999999, 0.210001]
 [0.02999999, 0.170001]

In [191]:
# 一対比較行列を1000個読み込み
df1 = CSV.File("./src/data/N5_a3_D_PCM_int.csv", header=false) |> DataFrame

# データフレームを5×5のサブデータフレームに分割
subdfs = split_dataframe(df1, 5)


1000-element Vector{Any}:
 5×5 DataFrame
 Row │ Column1  Column2  Column3  Column4   Column5 
     │ Float64  Float64  Float64  Float64   Float64 
─────┼──────────────────────────────────────────────
   1 │     1.0      1.0     2.0   2.0           5.0
   2 │     1.0      1.0     1.0   2.0           2.0
   3 │     0.5      1.0     1.0   2.0           4.0
   4 │     0.5      0.5     0.5   1.0           3.0
   5 │     0.2      0.5     0.25  0.333333      1.0
 5×5 DataFrame
 Row │ Column1   Column2  Column3  Column4  Column5 
     │ Float64   Float64  Float64  Float64  Float64 
─────┼──────────────────────────────────────────────
   1 │ 1.0           1.0      2.0      3.0      4.0
   2 │ 1.0           1.0      1.0      1.0      2.0
   3 │ 0.5           1.0      1.0      1.0      1.0
   4 │ 0.333333      1.0      1.0      1.0      1.0
   5 │ 0.25          0.5      1.0      1.0      1.0
 5×5 DataFrame
 Row │ Column1  Column2   Column3   Column4  Column5 
     │ Float64  Float64   Float64   F

In [195]:
# それぞれのデータで推定
for subdf in subdfs
    # 一対比較行列を取り出す
    A = Matrix(subdf)
    display(A)

    # 提案手法で区間重要度推定
    solution = solveIntervalAHP(A, EV)
    E = solution.W # 区間重要度の推定値
    # display(L"E = %$(intervalVectorLaTeXString(E))")
    # display(L"T = %$(intervalVectorLaTeXString(T))")

    TcapE = T .∩ E # 真の区間重要度と推定値の積集合
    # display(TcapE)
    TcupE = T .∪ E # 真の区間重要度と推定値の和集合
    # display(TcupE)

    P = Vector{Float64}()
    Q = Vector{Float64}()
    R = Vector{Float64}()
    F = Vector{Float64}()
    
    # c_diam は、空集合の場合は幅0を返す
    P = c_diam.(TcapE) ./ c_diam.(TcupE) # 真の区間重要度と推定値の積集合の割合
    Q = c_diam.(TcapE) ./ c_diam.(T) # 真の区間重要度と推定値の積集合の割合
    R = c_diam.(TcapE) ./ c_diam.(E) # 真の区間重要度と推定値の積集合の割合
    
    # F値を計算する関数(ゼロ割り回避のため)
    function calculate_F(Q, R)
        denominator = Q .+ R
        # 分母が 0 でない場合に計算を行う
        F = ifelse.(denominator .== 0, 0.0, 2 * (Q .* R) ./ denominator)
        return F
    end
    
    F = calculate_F(Q, R) # F値

    display(L"P = %$(VectorLaTeXString(P))")
    display(L"Q = %$(VectorLaTeXString(Q))")
    display(L"R = %$(VectorLaTeXString(R))")
    display(L"F = %$(VectorLaTeXString(F))")

end

5×5 Matrix{Float64}:
 1.0  1.0  2.0   2.0       5.0
 1.0  1.0  1.0   2.0       2.0
 0.5  1.0  1.0   2.0       4.0
 0.5  0.5  0.5   1.0       3.0
 0.2  0.5  0.25  0.333333  1.0

L"$P = \begin{pmatrix} 0.298 \\ 0.441 \\ 0.604 \\ 0.36 \\ 0.696 \end{pmatrix}$"

L"$Q = \begin{pmatrix} 1.0 \\ 1.0 \\ 1.0 \\ 0.839 \\ 0.921 \end{pmatrix}$"

L"$R = \begin{pmatrix} 0.298 \\ 0.441 \\ 0.604 \\ 0.387 \\ 0.74 \end{pmatrix}$"

L"$F = \begin{pmatrix} 0.46 \\ 0.612 \\ 0.753 \\ 0.53 \\ 0.821 \end{pmatrix}$"

...

5×5 Matrix{Float64}:
 1.0  1.0       1.0  1.0  2.0
 1.0  1.0       2.0  3.0  4.0
 1.0  0.5       1.0  1.0  5.0
 1.0  0.333333  1.0  1.0  2.0
 0.5  0.25      0.2  0.5  1.0

L"$P = \begin{pmatrix} 0.31 \\ 0.325 \\ 0.327 \\ 0.325 \\ 0.506 \end{pmatrix}$"

L"$Q = \begin{pmatrix} 1.0 \\ 1.0 \\ 0.871 \\ 0.845 \\ 0.98 \end{pmatrix}$"

L"$R = \begin{pmatrix} 0.31 \\ 0.325 \\ 0.343 \\ 0.345 \\ 0.511 \end{pmatrix}$"

L"$F = \begin{pmatrix} 0.473 \\ 0.49 \\ 0.492 \\ 0.49 \\ 0.672 \end{pmatrix}$"